# Image Classification

**Motivation.** In this section we will introduce the Image Classification problem, which is the task of assigning an input image one label from a fixed set of categories. This is one of the core problems in Computer Vision that, despite its simplicity, has a large variety of practical applications. Moreover, as we will see later in the course, many other seemingly distinct Computer Vision tasks (such as object detection, segmentation) can be reduced to image classification.

**Example.** For example, in the image below an image classification model takes a single image and assigns probabilities to 4 labels, {*cat, dog, hat, mug*}. As shown in the image, keep in mind that to a computer an image is represented as **one large 3-dimensional array of numbers**. 

In this example, the cat image is:
- 248 pixels wide 
- 400 pixels tall 
- 3 color channels Red, Green, Blue (or RGB for short). 

Therefore, the image consists of **248 x 400 x 3** numbers, or a total of **297,600** numbers. 

Each number is an integer that ranges from **0 (black) to 255 (white)**. 

Our task is to turn this quarter of a million numbers into a single label, such as “cat”.

![aaa](./Piks/classify.png)

*The task in Image Classification is to predict a single label (or a distribution over labels as shown here to indicate our confidence) for a given image. Images are 3-dimensional arrays of integers from 0 to 255, of size Width x Height x 3. The 3 represents the three color channels Red, Green, Blue.*

**Challenges**. Since this task of recognizing a visual concept (e.g. cat) is relatively trivial for a human to perform, it is worth considering the challenges involved from the perspective of a Computer Vision algorithm. As we present (an inexhaustive) list of challenges below, keep in mind the **raw representation of images as a 3-D array of brightness values**:
- **Viewpoint variation**. A single instance of an object can be oriented in many ways with respect to the camera.
- **Scale variation**. Visual classes often exhibit variation in their size (size in the real world, not only in terms of their extent in the image).
- **Deformation**. Many objects of interest are not rigid bodies and can be deformed in extreme ways.
- **Occlusion**. The objects of interest can be occluded. Sometimes only a small portion of an object (as little as few pixels) could be visible.
- **Illumination conditions**. The effects of illumination are drastic on the pixel level.
- **Background clutter**. The objects of interest may blend into their environment, making them hard to identify.
- **Intra-class variation**. The classes of interest can often be relatively broad, such as chair. There are many different types of these objects, each with their own appearance.

A good image classification model must be invariant to the cross product of all these variations, while simultaneously retaining sensitivity to the inter-class variations.

![](./Piks/challenges.jpeg)

**Data-driven approach**. 
<br/>How might we go about writing an algorithm that can classify images into distinct categories? 
<br/>Unlike writing an algorithm for, for example, sorting a list of numbers, it is not obvious how one might write an algorithm for identifying cats in images. Therefore, instead of trying to specify what every one of the categories of interest look like directly in code, the approach that we will take is not unlike one you would take with a child: we’re going to provide the computer with many examples of each class and then develop learning algorithms that look at these examples and learn about the visual appearance of each class. This approach is referred to as a **data-driven approach**, since it relies on first **accumulating a training dataset of labeled images**. Here is an example of what such a dataset might look like:

![](./Piks/trainset.jpg)

*An example training set for four visual categories. In practice we may have thousands of categories and hundreds of thousands of images for each category.*

**The image classification pipeline.** 
<br/>We’ve seen that the task in Image Classification is:
- Take an array of pixels that represents a single image 
- Assign a label to it. 

Our complete pipeline can be formalized as follows:
- **Input**: Our input consists of a set of N images, each labeled with one of K different classes. We refer to this data as the training set.
- **Learning**: Our task is to use the training set to learn what every one of the classes looks like. We refer to this step as training a classifier, or learning a model.
- **Evaluation**: In the end, we evaluate the quality of the classifier by asking it to predict labels for a new set of images that it has never seen before. We will then compare the true labels of these images to the ones predicted by the classifier. Intuitively, we’re hoping that a lot of the predictions match up with the true answers (which we call the ground truth).

## Nearest Neighbor Classifier

As our first approach, we will develop what we call a **Nearest Neighbor Classifier**. This classifier has *nothing to do with* *Convolutional Neural Networks* and it is very **rarely used in practice**, but it will allow us to *get an idea* about the basic approach to an image classification problem.

**Example image classification dataset: CIFAR-10.**
<br/>One popular toy image classification dataset is the CIFAR-10 dataset. This dataset consists of **60,000** tiny images that are **32 pixels high and wide**. Each image is labeled with **one of 10 classes** (for example *airplane, automobile, bird, etc*). 

These 60,000 images are partitioned into:
- A training set of 50,000 images 
- A test set of 10,000 images

In the image below you can see 10 random example images from each one of the 10 classes:

![](./Piks/nn.jpg)

*Left: Example images from the [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html). 
<br/>Right: first column shows a few test images and next to each we show the top 10 nearest neighbors in the training set according to pixel-wise difference.*

Suppose now that we are given the CIFAR-10 training set of 50,000 images (5,000 images for every one of the labels), and we wish to label the remaining 10,000. The nearest neighbor classifier will take a test image, compare it to every single one of the training images, and predict the label of the closest training image. In the image above and on the right you can see an example result of such a procedure for 10 example test images. 

Notice that in only about **3 out of 10** examples an image of the same class is retrieved, while in the other 7 examples this is not the case. For example, in the 8th row the nearest training image to the horse head is a red car, presumably due to the strong black background. As a result, this image of a horse would in this case be mislabeled as a car.

You may have noticed that we left unspecified the details of exactly how we compare two images, which in this case are just two blocks of 32 x 32 x 3. 

One of the simplest possibilities is to:
- compare the images pixel by pixel
- add up all the differences

In other words, given two images and representing them as vectors $I_1, I_2$, a reasonable choice for comparing them might be the **L1 distance**:

$$d_1 (I_1, I_2) = \sum_{p} \left| I^p_1 - I^p_2 \right|$$

Where the sum is taken over all pixels. Here is the procedure visualized:

![](./Piks/nneg.jpeg)

*An example of using pixel-wise differences to compare two images with L1 distance (for one color channel in this example). Two images are subtracted elementwise and then all differences are added up to a single number*:
- *If two images are identical the result will be zero.* 
- *But if the images are very different the result will be large.*

**More information about distance formular**: [here](./Additional_info/Distance_Formula.ipynb)

In [13]:
#code to illustrate the picture above
import numpy as np

test_image = np.array([ [56, 32,  10, 18],
                        [90, 23, 128, 133],
                        [24, 26, 178, 200],
                        [ 2,  0, 255, 220]])
training_image = np.array([ [10, 20,  24,  17],
                            [ 8, 10,  89, 100],
                            [12, 16, 178, 170],
                            [ 4, 32, 233, 112]])
diff = np.abs(test_image - training_image)
diff

array([[ 46,  12,  14,   1],
       [ 82,  13,  39,  33],
       [ 12,  10,   0,  30],
       [  2,  32,  22, 108]])

In [14]:
diff.sum()

456

Let’s also look at how we might implement the classifier in code. 
<br/>First, let’s load the CIFAR-10 data into memory as **4 arrays**: 
- The training data/labels: Xtr, Ytr
- The test data/labels: Xte, Yte

In the code below:
- Xtr (of size 50,000 x 32 x 32 x 3) holds all the images in the training set
- Ytr, a corresponding 1-dimensional array (of length 50,000) holds the training labels (from 0 to 9):

In [ ]:
Xtr, Ytr, Xte, Yte = load_CIFAR10('data/cifar10/') # a magic function we provide
# flatten out all images to be one-dimensional
Xtr_rows = Xtr.reshape(Xtr.shape[0], 32 * 32 * 3) # Xtr_rows becomes 50000 x 3072
Xte_rows = Xte.reshape(Xte.shape[0], 32 * 32 * 3) # Xte_rows becomes 10000 x 3072

Now that we have all images stretched out as rows, here is how we could train and evaluate a classifier:

In [ ]:
nn = NearestNeighbor() # create a Nearest Neighbor classifier class
nn.train(Xtr_rows, Ytr) # train the classifier on the training images and labels
Yte_predict = nn.predict(Xte_rows) # predict labels on the test images
# and now print the classification accuracy, which is the average number
# of examples that are correctly predicted (i.e. label matches)
print('accuracy: %f' % (np.mean(Yte_predict == Yte)))

Notice that as an evaluation criterion, it is common to use the **accuracy**, which measures the **fraction of predictions that were correct**. 

**More information about Accuracy, Recall, Precision, F1 Score**: [here](./Additional_info/Accuracy_Recall_Precision_and_F1Score.ipynb)

Notice that all classifiers we will build satisfy this one common API: 
- They have a `train(X,y)` function that takes the data and the labels to learn from. 
- Internally, the class should build some kind of model of the labels and how they can be predicted from the data. 
- There is a `predict(X)` function, which takes new data and predicts the labels. 

Of course, we’ve left out the meat of things - the actual classifier itself. Here is an implementation of a simple Nearest Neighbor classifier with the L1 distance that satisfies this template:

In [ ]:
import numpy as np

class NearestNeighbor(object):
    def __init__(self):
        pass

    def train(self, X, y):
        """ X is N x D where each row is an example. Y is 1-dimension of size N """
        # the nearest neighbor classifier simply remembers all the training data
        self.Xtr = X
        self.ytr = y

    def predict(self, X):
        """ X is N x D where each row is an example we wish to predict label for """
        num_test = X.shape[0]
        # lets make sure that the output type matches the input type
        Ypred = np.zeros(num_test, dtype = self.ytr.dtype)

        # loop over all test rows
        for i in range(num_test):
            # find the nearest training image to the i'th test image
            # using the L1 distance (sum of absolute value differences)
            distances = np.sum(np.abs(self.Xtr - X[i,:]), axis = 1)
            min_index = np.argmin(distances) # get the index with smallest distance
            Ypred[i] = self.ytr[min_index] # predict the label of the nearest example

        return Ypred